In [1]:
#!pip install -q torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 torchtext==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
!nvidia-smi

Wed Apr 21 10:12:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 207... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   54C    P0    63W / 215W |    442MiB /  8192MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [3]:
#!pip install -q --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install torch-geometric

In [4]:
#!pip install optuna

In [5]:
import torch
import optuna
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from sklearn.neural_network import MLPClassifier 
from sklearn.linear_model import LogisticRegressionCV

from sklearn.ensemble import GradientBoostingClassifier
from torch_geometric.data import NeighborSampler
from torch.optim import lr_scheduler
import torch.optim as optim
import matplotlib.pyplot as plt
import collections
import numpy as np
from modules.model import Net
from sklearn.metrics import precision_score
from datetime import datetime
from modules.sampling import Sampler
from datetime import datetime

In [6]:
import pickle
import os
class Main():
    def __init__(self,conv, device, loss_function, mode = 'unsupervised',**kwargs):
        self.dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
        data = self.dataset[0]
        self.Conv = conv
        self.device = device# if torch.cuda.is_available() else 'cpu')
        self.x = data.x#.to(device)
        self.y = data.y.squeeze()#.to(device)
        self.data=data.to(device)
        self.loss = loss_function
        self.mode = mode
        self.train_mask = torch.tensor([True]*int(0.8*len(data.x)+1) + [False]*int(0.2*len(data.x)))
        #val_mask = torch.tensor([False]*int(0.6*len(data.x)+1) + [True]*int(0.2*len(data.x)+1)+[False]*int(0.2*len(data.x)))
        self.test_mask = torch.tensor([False]*int(0.8*len(data.x)+1) + [True]*int(0.2*len(data.x)))
        self.flag = self.loss["flag"]
        
      #  if self.flag:
       #     name_of_file = "samples_"+self.loss["Name"]+".pickle"
        #    if os.path.exists(name_of_file):
         #       with open(name_of_file,'rb') as f:
          #          self.samples = pickle.load(f)
           # else:
            #    arr=torch.nonzero(self.train_mask == True)
             #   indices_of_train_data = ([item for sublist in arr for item in sublist])
              #  self.samples = self.LossSampler.sample(indices_of_train_data,alpha = self.alpha)

        super(Main, self).__init__()
    
    def train(self, model,data,optimizer,Sampler,train_loader,dropout,epoch,**kwargs):
        model.train()   
        total_loss = 0
        optimizer.zero_grad()
        if model.mode == 'unsupervised':
            if model.conv=='GCN':
                arr=torch.nonzero(self.train_mask == True)
                indices_of_train_data = ([item for sublist in arr for item in sublist])
                d_out = datetime.now()
                out = model.inference(data.to(device),dp=dropout)
                #print('свертка посчиталась за: ',datetime.now() - d_out)
                d_samples=datetime.now()
                if (self.flag and epoch == 0) or not self.flag:
                    self.samples = Sampler.sample(indices_of_train_data) 

                #print('сэмплирование соседей для ф.п./матрица similarity: ', datetime.now()-d_samples)

                d_loss = datetime.now()
                loss = model.loss(out[self.train_mask], self.samples)
                #print('подсчет функции потерь: ', datetime.now()-d_loss)
                total_loss+=loss
            else:
                for batch_size, n_id, adjs in train_loader:
                    if len(train_loader.sizes) == 1:
                        adjs = [adjs]
                    # adjs holds a list of (edge_index, e_id, size) tuples.
                    adjs = [adj.to(device) for adj in adjs]
                    d_out = datetime.now()
                    out = model.forward(data.x[n_id.to(device)].to(device), adjs)
                   # print('свертка (на батче) посчиталась за: ',datetime.now() - d_out)
                    d_samples=datetime.now()
                    if (self.flag and epoch == 0) or not self.flag:
                        self.samples = Sampler.sample(n_id[:batch_size]) 

                                          
                    #print('сэмплирование соседей для ф.п./матрица similarity: ', datetime.now()-d_samples)
                    d_loss = datetime.now()
                    loss = model.loss(out, self.samples)#pos_batch.to(device), neg_batch.to(device))
                    #print('подсчет функции потерь: ', datetime.now()-d_loss)
                    total_loss+=loss
            total_loss.backward()
            optimizer.step()      
            return total_loss /len(train_loader)
        elif model.mode== 'supervised':
            if model.conv=='GCN':
                out = model.inference(data.to(device),dp=dropout)
                loss = model.loss_sup(out[self.train_mask],y[self.train_mask])
                total_loss+=loss
            else:
                for batch_size, n_id, adjs in train_loader:
                    adjs = [adj for adj in adjs]
                    out = model.forward(data.x[n_id].to(device), adjs)
                    loss = model.loss_sup(out,y[n_id[:batch_size]])
                    total_loss+=loss
            total_loss.backward(retain_graph=True)
            optimizer.step()      
            return total_loss /len(train_loader)       

    @torch.no_grad()
    def test(self, model,data,classifier,**kwargs):#,n_estimators,learning_rate_carboost, max_depth): 
        model.eval()
        out = model.inference(data.to(device))
        y_true = self.y.detach().numpy()
        if model.mode == 'supervised':
            y_true = self.y.unsqueeze(-1)
            y_pred = out.cpu().argmax(dim=-1, keepdim=True)
            accs = []
            for mask in [self.train_mask, self.val_mask, self.test_mask]:    
                accs+=[int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]
            return accs
        elif model.mode == 'unsupervised': 
            if classifier == 'logistic regression':
                clf = LogisticRegressionCV(cv = 5, max_iter = 3000).fit(out.cpu().detach()[self.train_mask].numpy(), self.y.detach()[self.train_mask].numpy())
            else:
                n_estimators= kwargs["n_estimators"]
                learning_rate_catboost = kwargs["learning_rate_catboost"]
                max_depth = kwargs["max_depth"]
                clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate = learning_rate_catboost, max_depth=max_depth, random_state=0)
                clf.fit(out.cpu().detach()[self.train_mask].numpy(), self.y.detach()[self.train_mask].numpy())
            accs = []
            for mask in [self.train_mask, self.test_mask]:    
                accs += [(clf.score(out.cpu().detach()[mask].numpy(), self.y.detach()[mask].cpu().numpy()))]
            return accs
            

    def run(self,**kwargs):
        
        hidden_layer = 64
        out_layer = 128
        dropout = 0.4
        size = 1
        learning_rate = 0.001

        classifier = "logistic regression"
        train_loader = NeighborSampler(self.data.edge_index, node_idx=self.train_mask, batch_size = int(sum(self.train_mask)), sizes=[-1]*size)
        LossSampler = Sampler(self.data,device=device,mask=self.train_mask,loss_info=self.loss)
        model = Net(dataset = self.dataset,mode=self.mode,conv=self.Conv,loss_function=self.loss,device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = (size),dropout = dropout)
        model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)
                #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.01, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
        scheduler=lr_scheduler.StepLR(optimizer, step_size=25,gamma=0.1)
        losses=[]
        train_accs=[]
        test_accs=[]
        val_accs=[]
        name_of_plot='conv: '+model.conv+', mode: '+model.mode+', loss from '+self.loss["Name"]

        print(name_of_plot)

        for epoch in range(50):
                    print('epoch',epoch)
                    loss = self.train(model,self.data,optimizer,LossSampler,train_loader,0,0.7, epoch)
                    losses.append(loss)
                    d_test = datetime.now()
                    train_acc, test_acc = self.test(model,self.data,'logistic regression')
                   # print('тестирование заняло: ', datetime.now()-d_test)
                    train_accs.append(train_acc)
                    test_accs.append(test_acc)
                   # val_accs.append(val_acc)
                    log = 'Loss: {:.4f}, Epoch: {:03d}, Train: {:.4f}, Test: {:.4f}'
                    #scheduler.step()
                    print(log.format(loss, epoch, train_acc, test_acc))
        print('Test acc on the last epoch ', test_acc)
        plt.plot(losses)
        plt.title(name_of_plot+' loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()
        plt.plot(test_accs)
        plt.title(name_of_plot+' loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()
                    #return [precision_score(y_true[train_mask.cpu()], best_preds_train, average='macro'), precision_score(y_true[val_mask.cpu()], best_preds_val, average='macro'),precision_score(y_true[test_mask.cpu()], best_preds, average='macro')]

In [7]:
class MainOptuna(Main):
    def objective(self,trial):
        # Integer parameter
        hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
        out_layer = trial.suggest_categorical("out_layer", [32,64,128])
        dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
        size = trial.suggest_categorical("size of network, number of convs", [1,2,3])
        Conv = self.Conv# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
        if self.loss["Name"] == "SAGE" or self.loss["Name"] == "DeepWalk" or self.loss["Name"] == "Node2Vec":
            walk_length = trial.suggest_int("walk length",5,20,step = 5)
            walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
            context_size = trial.suggest_int("context size",5,20,step = 5)
        if self.loss["Name"] == "Node2Vec":
            q = trial.suggest_float("q", 0.0,0.9,step = 0.1)
            p = trial.suggest_float("p", 0.0,0.9,step = 0.1)
        if self.loss["C"] == "RPR" or self.loss["C"] == "PPR":
            alpha = trial.suggest_float("alpha", 0.5,1)

        num_negative_samples = trial.suggest_int("num_negative_samples",1,16,step = 5)
        self.loss =  {"Name": "VERSE_Adj","C": "Adj","num negative samples":num_negative_samples,"loss var": "Context Matrix","flag":True} 

        model = Net(dataset = self.dataset,mode='unsupervised',conv=Conv,loss_function=self.loss,device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = size,dropout = dropout)

        train_loader = NeighborSampler(self.data.edge_index, batch_size = len(self.train_mask),node_idx=self.train_mask, sizes=[-1]*size)
        LossSampler = Sampler(self.data,device=self.device,mask=self.train_mask,loss_info=self.loss)

        model.to(device)

    #optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
        learning_rate= trial.suggest_float("lr",5e-4,1e-2)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  

        classifier = "logistic regression" #trial.suggest_categorical("classifier", ["logistic regression", "catboost"])

        if classifier == "catboost":
            n_estimators = trial.suggest_int("n of estimators", 10,40,5)
            learning_rate_catboost = trial.suggest_float("lr_catboost",5e-4,1e-2)
            max_depth = trial.suggest_int("max_depth",1,10,2)
        else:
            n_estimators = -1
            learning_rate_catboost =-1
            max_depth = -1
        #training of the model
        for epoch in range(50):
            loss = self.train(model,self.data,optimizer,LossSampler,train_loader,dropout,epoch)

        train_acc, test_acc = self.test(model,self.data,classifier,n_estimators=n_estimators,learning_rate_catboost=learning_rate_catboost,max_depth=max_depth)


        trial.report(test_acc,epoch)

        return test_acc

    
    def run(self):


        self.Conv = "GCN"
        study = optuna.create_study(direction="maximize",study_name=self.loss["Name"]+" loss, GCN conv")
        study.optimize(self.objective,n_trials = 10)

        print('Best trial:')
        trial = study.best_trial
        print(" Value: ", trial.value)
        print(" Params: ")
        for key, value in trial.params.items():
            print(" {}: {}".format(key,value))
        
        self.Conv = "SAGE"
        study = optuna.create_study(direction="maximize",study_name=self.loss["Name"]+" loss, SAGE conv")
        study.optimize(self.objective,n_trials = 10)

        print('Best trial:')
        trial = study.best_trial
        print(" Value: ", trial.value)
        print(" Params: ")
        for key, value in trial.params.items():
            print(" {}: {}".format(key,value))
        
        self.Conv = "GAT"
        study = optuna.create_study(direction="maximize",study_name=self.loss["Name"]+" loss, GAT conv")
        study.optimize(self.objective,n_trials = 10)

        print('Best trial:')
        trial = study.best_trial
        print(" Value: ", trial.value)
        print(" Params: ")
        for key, value in trial.params.items():
            print(" {}: {}".format(key,value))    
            
        
      

In [8]:
SAGE = {"Name":"SAGE" , "walk length":5,"walks per node":50,"num negative samples":20,"context size" : 10,"p":1,"q":1, "loss var": "Random Walks","flag":False}
DeepWalk = {"Name": "DeepWalk","walk length":10,"walks per node":10,"num negative samples":10,"context size" : 10,"p":1,"q":1,"loss var": "Random Walks","flag":False } #Проблемы с памятью после того, как увеличила количество тренировочных данных
#DeepWalk = {"Name": "DeepWalk","walk length":40,"walks per node":80,"num negative samples":20,"context size" : 10,"p":1,"q":1,"loss var": "Random Walks" } #Проблемы с памятью после того, как увеличила количество тренировочных данных
Node2Vec = {"Name": "Node2Vec","walk length":20,"walks per node":10,"num negative samples":1,"context size" : 10,"p":1.414 ,"q":1.414, "loss var": "Random Walks","flag":False}#то же самое 
#Node2Vec = {"Name": "Node2Vec","walk length":100,"walks per node":18,"num negative samples":20,"context size" : 16,"p":1.414 ,"q":1.414, "loss var": "Random Walks"}#то же самое 

LINE = {"Name": "LINE","C": "Adj","num negative samples":10,"loss var": "Context Matrix","flag":True} 
HOPE_RPR = {"Name": "HOPE_RPR","C":"RPR","loss var": "Factorization","flag":True} #проверить
HOPE_Katz = {"Name": "HOPE_Katz","C":"Katz","loss var": "Factorization","flag":True} #проверить
HOPE_CN = {"Name": "HOPE_CommonNeighbors","C":"CN","loss var": "Factorization","flag":True} 
HOPE_AA = {"Name": "HOPE_AdamicAdar","C":"AA","loss var": "Factorization","flag":True} 

LapEigen = {"Name": "LaplacianEigenMaps", "C":"Adj","loss var": "Laplacian EigenMaps","flag":True}

GraphFactorization = {"Name": "Graph Factorization","C":"Adj","loss var": "Factorization","flag":True} 
VERSE_PPR =  {"Name": "VERSE_PPR","C": "PPR","num negative samples":10,"loss var": "Context Matrix","flag":True}

VERSE_SR =  {"Name": "VERSE_SimRank","C": "SR","num negative samples":10,"loss var": "Context Matrix","flag":True} 
VERSE_Adj =  {"Name": "VERSE_Adj","C": "Adj","num negative samples":10,"loss var": "Context Matrix","flag":True} 

APP ={}#Implement
Struc2Vec ={} #Implement

In [ ]:
device = torch.device('cuda')
MO = MainOptuna('SAGE', device, VERSE_Adj, mode = 'unsupervised')
MO.run()

[I 2021-04-21 10:12:36,079] A new study created in memory with name: VERSE_Adj loss, GAT conv
[I 2021-04-21 10:14:10,908] Trial 0 finished with value: 0.8022181146025879 and parameters: {'hidden_layer': 32, 'out_layer': 32, 'dropout': 0.5, 'size of network, number of convs': 1, 'num_negative_samples': 11, 'lr': 0.0037438473939578614}. Best is trial 0 with value: 0.8022181146025879.
[I 2021-04-21 10:15:47,695] Trial 1 finished with value: 0.8133086876155268 and parameters: {'hidden_layer': 64, 'out_layer': 64, 'dropout': 0.4, 'size of network, number of convs': 1, 'num_negative_samples': 6, 'lr': 0.009329630500344502}. Best is trial 1 with value: 0.8133086876155268.
[I 2021-04-21 10:17:07,875] Trial 2 finished with value: 0.3179297597042514 and parameters: {'hidden_layer': 128, 'out_layer': 64, 'dropout': 0.1, 'size of network, number of convs': 2, 'num_negative_samples': 16, 'lr': 0.006806163697828752}. Best is trial 1 with value: 0.8133086876155268.
[I 2021-04-21 10:18:40,541] Trial 3

Best trial:
 Value:  0.8243992606284658
 Params: 
 hidden_layer: 64
 out_layer: 64
 dropout: 0.4
 size of network, number of convs: 1
 num_negative_samples: 16
 lr: 0.008293519025088082


[I 2021-04-21 10:29:08,410] Trial 0 finished with value: 0.7966728280961183 and parameters: {'hidden_layer': 128, 'out_layer': 64, 'dropout': 0.5, 'size of network, number of convs': 1, 'num_negative_samples': 1, 'lr': 0.007662113449191953}. Best is trial 0 with value: 0.7966728280961183.
[I 2021-04-21 10:30:37,666] Trial 1 finished with value: 0.6469500924214417 and parameters: {'hidden_layer': 128, 'out_layer': 32, 'dropout': 0.1, 'size of network, number of convs': 1, 'num_negative_samples': 6, 'lr': 0.0009343452769613199}. Best is trial 0 with value: 0.7966728280961183.
[I 2021-04-21 10:32:18,307] Trial 2 finished with value: 0.5508317929759704 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 2, 'num_negative_samples': 16, 'lr': 0.007985304254885993}. Best is trial 0 with value: 0.7966728280961183.
[I 2021-04-21 10:33:52,279] Trial 3 finished with value: 0.7689463955637708 and parameters: {'hidden_layer': 256, 'out_layer': 

Best trial:
 Value:  0.8022181146025879
 Params: 
 hidden_layer: 256
 out_layer: 64
 dropout: 0.5
 size of network, number of convs: 1
 num_negative_samples: 11
 lr: 0.009823445521725012


[I 2021-04-21 10:44:41,452] Trial 0 finished with value: 0.5804066543438078 and parameters: {'hidden_layer': 32, 'out_layer': 32, 'dropout': 0.5, 'size of network, number of convs': 3, 'num_negative_samples': 6, 'lr': 0.0025622630013137936}. Best is trial 0 with value: 0.5804066543438078.
[I 2021-04-21 10:46:05,734] Trial 1 finished with value: 0.3179297597042514 and parameters: {'hidden_layer': 32, 'out_layer': 64, 'dropout': 0.2, 'size of network, number of convs': 2, 'num_negative_samples': 1, 'lr': 0.00876622933238325}. Best is trial 0 with value: 0.5804066543438078.
[I 2021-04-21 10:47:26,422] Trial 2 finished with value: 0.3179297597042514 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.0, 'size of network, number of convs': 3, 'num_negative_samples': 1, 'lr': 0.006966361252059097}. Best is trial 0 with value: 0.5804066543438078.
